In [19]:
import scipy.io
import numpy as np
import networkx as nx
import cvxpy as cp
def Get_NetInfo(T, N_node):
    # 初始化路径矩阵K，大小为N_node x N_arc
    N_arc = T.shape[0]
    K = np.zeros((N_node, N_arc))

    # 填充路径矩阵K
    for i in range(N_node):
        for j in range(N_arc):
            if T[j, 0] == i:
                K[i, j] = -1
            if T[j, 1] == i:
                K[i, j] = 1

    return K

# 示例用法：
# 假设T是一个N_arc x 3的NumPy数组，其中每一行代表一个边，格式为[起点, 终点, 边权]
# N_node是节点的数量
# T = np.array([[1, 2, 1], [2, 3, 1], [3, 1, 1]])  # 示例网络边信息
# N_node = 3  # 示例节点数量
# K = Get_NetInfo(T, N_node)
# print(K)

import numpy as np
from scipy.sparse import csr_matrix


def get_V2Einfo(T, N_node):
    # 网络预处理程序，得到 C:边权向量；K：路径矩阵；D：节点与边转换矩阵；A：邻接矩阵

    N_arc = T.shape[0]

    # 初始化稀疏矩阵
    A = csr_matrix((N_node, N_node), dtype=int)
    K = csr_matrix((N_node, N_arc), dtype=int)
    D = csr_matrix((N_arc, N_node), dtype=int)
    C = csr_matrix((N_arc, 1), dtype=int)

    # 得到邻接矩阵A
    for i in range(N_arc):
        A[int(T[i, 0] - 1), int(T[i, 1] - 1)] = 1  # Python 索引从0开始，所以需要减1

    # 得到路径矩阵K
    for i in range(N_node):
        for j in range(N_arc):
            if T[j, 0] == i + 1:  # Python 索引从0开始，所以需要加1
                K[i, j] = -1
            if T[j, 1] == i + 1:  # Python 索引从0开始，所以需要加1
                K[i, j] = 1

    # 得到节点与边转换矩阵D
    for i in range(N_arc):
        D[i, int(T[i, 0] - 1)] = 1  # Python 索引从0开始，所以需要减1

    # 得到边权向量C
    for i in range(N_arc):
        C[i, 0] = T[i, 2]

    #return D, A, K, C
    return D

# 示例使用T和N_node的代码
# 假设T是一个N_arc x 3的NumPy数组，其中每行包含三个元素（起点，终点，权重）
# N_node是节点的数量
# T = np.array([[1, 2, 10], [2, 3, 15], ...])  # 示例输入
# D, A, K, C = get_V2Einfo(T, N_node)'''




# Load data from .mat files
congo_net = scipy.io.loadmat('CongoNet.mat')
congo_net_node_location = scipy.io.loadmat('CongoNetNodeLocation.mat')

N_arc = len(congo_net['CongoNet'][:,0])
N_node = len(congo_net_node_location['CongoNetNodeLocation'][:,0])

# Load parameters
demand = scipy.io.loadmat('Demand.mat')
h = scipy.io.loadmat('h.mat')

n = len(demand['Demand'][0,:])  # t=1,2,...,n=222
L = np.zeros((N_node, N_node))
snode = congo_net['CongoNet'][:, 0]
tnode = congo_net['CongoNet'][:, 1]
weights = congo_net['CongoNet'][:, 2]
h=h['h'][:,:]
# Create digraph for CongoNet
Gnet = nx.DiGraph()
Gnet.add_weighted_edges_from(zip(snode, tnode, weights))
L = nx.floyd_warshall_numpy(Gnet)

q = 1*np.random.rand(N_arc, 1)

Demand = demand['Demand'][:, :n]
og = np.array(Demand, dtype='float32')
og[og != 0] = 1
ot = 107
og[ot, :] = -1

# Get NetInfo and V2Einfo
K = Get_NetInfo(congo_net['CongoNet'], N_node)
T = get_V2Einfo(congo_net['CongoNet'], N_node)
c = congo_net['CongoNet'][:, 2]
d = 1 + np.round(np.mean(c) * np.random.rand(N_arc, 1))

w = np.sum(demand['Demand'], 1)

N_I = np.zeros((n, 1))
f = []
a=np.sum(h,axis=0)

In [20]:
for t in range(n):
    N_I[t,0] = a[t]
    f.append(1 + np.round(3 * np.mean(c) * np.random.rand(int(N_I[t, 0]), 1)))
a = []
r = []
x = []
xoff = []

# Multi-stage game
p = [0,0]

for t in range(n):
    #print(t)
    Z = cp.Variable((N_node, int(N_I[t, 0])), boolean=True)
    Za = cp.Variable((N_arc, 1))
    wi = cp.Variable((N_node, 1))
    c=c.reshape(2188,1)
    temp11=(Z.T @ (L @ h[:, t])).size
    #print(temp11)
    # Constraints
    Cons = [K.T @ wi <= (c + cp.multiply(d , Za)), wi[0] == 0, Z.T @ np.ones((N_node,1)) <= 1, Z.T @ (L @ h[:, t]) <= f[t].reshape(temp11,),
                   0 <= Z, Z <= 1, Za == T @ cp.sum(Z, axis=1,keepdims=True)]
        # Objective
    ut = og[:, t].T @ wi


    # Solve
    prob = cp.Problem(cp.Minimize(ut), Cons)
    result = prob.solve()

    a.append(ut.value)

In [21]:
f2=open('tosee/a.json', 'w',encoding = 'utf-8')
for km in range(len(a)):
    f2.write(str(a[km])+'\n')
f2.close()
